### Développement du "Modèle avancé BERT"
Ce notebook implémente un modèle de classification de sentiments utilisant BERT. 
Nous allons fine-tuner un modèle pré-entraîné sur notre jeu de données, puis évaluer ses performances et enregistrer les résultats avec MLFlow.


In [1]:
import pandas as pd

# Charger les données
data_path = "../data/training.1600000.processed.noemoticon.csv"
data = pd.read_csv(data_path, encoding='latin-1', header=None)

In [2]:
# Afficher un échantillon des données
data.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
data_sample = data[['target', 'text']].sample(5)
display(data_sample)

,target,text
171132,0,i need to get my computer fixed
266039,0,is f-ing exhausted. Don't think I'm gonna make...
906655,4,@dannisaywhat LOL kk i've sent u a friend requ...
1188597,4,@Dannymcfly hey we brazilians have amazing ass...
799186,0,@AgentBooth what happened?


In [3]:
# Transformer les valeurs de 'target' : 0 reste 0 (négatif) et 4 devient 1 (positif)
data['target'] = data['target'].apply(lambda x: 1 if x == 4 else 0)

# Vérifier un échantillon après la transformation
data_sample = data[['target', 'text']].sample(5)
display(data_sample)


,target,text
345755,0,@stefii_wefii ...a sleeping problem because I ...
606702,0,I can haz phone! Crap! Now I'm &quot;reachable...
881348,1,"@TheGeniousphere I think,you are already know ..."
1443971,1,@drewryanscott move hereee lol i think you'll ...
1322176,1,@SkoutTradeFair oh no! Poor you!


In [4]:
!pip install transformers

In [5]:
import pandas as pd
from transformers import BertTokenizer

# Définir la taille de l'échantillon
sample_size = 1000

# Limiter le dataset à un échantillon stratifié de 10000 lignes basé sur la colonne 'target'
sample_data = data.groupby('target', group_keys=False).apply(lambda x: x.sample(int(sample_size * len(x) / len(data)), random_state=42))

# Vérifier la distribution des classes dans l'échantillon par rapport au dataset d'origine
print("Distribution des classes dans le dataset d'origine :")
print(data['target'].value_counts(normalize=True))
print("\nDistribution des classes dans l'échantillon :")
print(sample_data['target'].value_counts(normalize=True))

# Prétraitement des données textuelles sur cet échantillon
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_text(text):
    tokens = tokenizer(text, max_length=128, truncation=True, padding='max_length', return_tensors='tf')
    return tokens

# Appliquer la fonction de prétraitement
sample_data['tokens'] = sample_data['text'].apply(preprocess_text)

# Afficher un échantillon des données après tokenisation
sample_data_sample = sample_data[['target', 'tokens']].sample(5)
display(sample_data_sample)


C:\Users\pat\.conda\envs\P7te\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\pat\AppData\Local\Temp\ipykernel_19076\3146143680.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_data = data.groupby('target', group_keys=False).apply(lambda x: x.sample(int(sample_size * len(x) / len(data)), random_state=42))


Distribution des classes dans le dataset d'origine :
target
0    0.5
1    0.5
Name: proportion, dtype: float64

Distribution des classes dans l'échantillon :
target
0    0.5
1    0.5
Name: proportion, dtype: float64


,target,tokens
384253,0,"[input_ids, token_type_ids, attention_mask]"
664875,0,"[input_ids, token_type_ids, attention_mask]"
735817,0,"[input_ids, token_type_ids, attention_mask]"
844006,1,"[input_ids, token_type_ids, attention_mask]"
1498998,1,"[input_ids, token_type_ids, attention_mask]"


### Charger et préparer le modèle BERT :

In [6]:
# Chargement et préparation du modèle BERT pré-entraîné
from transformers import TFBertForSequenceClassification
import tensorflow as tf

# Charger le modèle BERT pré-entraîné
model_bert = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compilations du modèle
model_bert.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Préparer les données et entraîner le modèle 

In [7]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

# Extract necessary elements for BERT (input_ids and attention_mask)
input_ids = np.array([t['input_ids'].numpy()[0] for t in sample_data['tokens']])
attention_masks = np.array([t['attention_mask'].numpy()[0] for t in sample_data['tokens']])
y = np.array(sample_data['target'].values)

# Split the data using train_test_split
X_train_ids, X_test_ids, X_train_mask, X_test_mask, y_train, y_test = train_test_split(
    input_ids, attention_masks, y, test_size=0.2, random_state=42
)

# Convert back to TensorFlow tensors
X_train_ids = tf.convert_to_tensor(X_train_ids)
X_test_ids = tf.convert_to_tensor(X_test_ids)
X_train_mask = tf.convert_to_tensor(X_train_mask)
X_test_mask = tf.convert_to_tensor(X_test_mask)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Prepare TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": X_train_ids, "attention_mask": X_train_mask}, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": X_test_ids, "attention_mask": X_test_mask}, y_test)).batch(32)

# Train the model
history = model_bert.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=3
)


Epoch 1/3


25/25 [==============================] - 305s 11s/step - loss: 0.6721 - accuracy: 0.5938 - val_loss: 0.5962 - val_accuracy: 0.6850
Epoch 2/3
25/25 [==============================] - 233s 9s/step - loss: 0.4995 - accuracy: 0.8050 - val_loss: 0.5083 - val_accuracy: 0.7650
Epoch 3/3
25/25 [==============================] - 236s 9s/step - loss: 0.3321 - accuracy: 0.8687 - val_loss: 0.4914 - val_accuracy: 0.7950


In [8]:
# Évaluation des performances
from sklearn.metrics import classification_report

# Préparer les données de test dans le bon format
test_inputs = {"input_ids": X_test_ids, "attention_mask": X_test_mask}

# Prédictions
y_pred = model_bert.predict(test_inputs)
y_pred_labels = tf.argmax(y_pred.logits, axis=1).numpy()

# Afficher le rapport de classification
print(classification_report(y_test, y_pred_labels))


7/7 [==============================] - 20s 2s/step
              precision    recall  f1-score   support

           0       0.82      0.74      0.78        96
           1       0.78      0.85      0.81       104

    accuracy                           0.80       200
   macro avg       0.80      0.79      0.79       200
weighted avg       0.80      0.80      0.79       200



### D'après le rapport de classification, voici les observations :

### Précision, rappel et F1-score par classe :
- **Classe 0** :
  - Précision : **0.82**
  - Rappel : **0.74**
  - F1-score : **0.78**
  - **Analyse** : Le modèle détecte correctement 82 % des exemples négatifs qu'il prédit, mais identifie seulement 74 % des véritables exemples négatifs.

- **Classe 1** :
  - Précision : **0.78**
  - Rappel : **0.85**
  - F1-score : **0.81**
  - **Analyse** : Le modèle est légèrement moins précis pour cette classe (78 % des exemples positifs prédits sont corrects), mais il détecte 85 % des véritables exemples positifs.

### Moyennes globales :
- **Accuracy (Exactitude)** : **0.80**, indiquant que 80 % des prédictions totales sont correctes.
- **Macro avg** : Moyennes arithmétiques des scores (précision, rappel, F1-score) pour les deux classes, autour de **0.79**.
- **Weighted avg** : Moyennes pondérées, tenant compte de la proportion de chaque classe, également autour de **0.79**.

### Interprétation :
Le modèle montre une performance globale correcte, avec un **F1-score moyen de 0.79**. La classe 1 (positif) est mieux détectée que la classe 0 (négatif), mais des marges d'amélioration subsistent, notamment sur le rappel de la classe 0.

### Recommandations pour l'amélioration :
- **Optimisation des hyperparamètres** : Ajuster le taux d'apprentissage, augmenter les époques ou expérimenter avec des optimisateurs différents.
- **Data augmentation** : Augmenter la diversité du dataset pour améliorer la généralisation du modèle.
- **Exploration de modèles alternatifs** : Tester des variantes de BERT ou fine-tuner davantage pour cette tâche spécifique.

### Conclusion :
Les résultats sont solides avec une **accuracy de 80 %** et des scores équilibrés entre les deux classes. Cependant, travailler sur le rappel de la classe 0 et tester des ajustements pourrait encore améliorer les performances.
```

In [9]:
from pathlib import Path
import mlflow
import mlflow.keras

# Chemin absolu pour le dossier "mlruns" en utilisant pathlib
mlruns_path = Path("../mlruns").resolve()

# Vérifier que le dossier "mlruns" existe, sinon le créer
if not mlruns_path.exists():
    mlruns_path.mkdir(parents=True)

# Vérifier que le sous-dossier ".trash" existe, sinon le créer
trash_folder = mlruns_path / ".trash"
if not trash_folder.exists():
    trash_folder.mkdir(parents=True)

# Configuration du chemin pour stocker les artefacts de MLFlow
mlflow.set_tracking_uri(mlruns_path.as_uri())

# Créer une nouvelle expérience ou utiliser une existante
experiment_name = "BERT_Embedding_Experiment"
mlflow.set_experiment(experiment_name)

# Démarrer une nouvelle session MLFlow pour BERT
with mlflow.start_run(nested=True):  # Utilisation d'une run imbriquée pour préserver les autres runs
    mlflow.log_param("model", "BERT")
    mlflow.log_param("epochs", 3)
    mlflow.log_metric("accuracy", history.history['accuracy'][-1])
    mlflow.log_metric("val_accuracy", history.history['val_accuracy'][-1])
    
    # Enregistrer le modèle BERT comme artefact
    mlflow.keras.log_model(model_bert, "model_bert")

print(f"Modèle BERT enregistré dans {mlruns_path}.")


2024/10/22 08:04:53 INFO mlflow.tracking.fluent: Experiment with name 'BERT_Embedding_Experiment' does not exist. Creating a new experiment.
2024/10/22 08:04:53 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:


Modèle BERT enregistré dans D:\OC_IA\P7\OC_IA_P7_analyse_sentiments_deep_Learning\mlruns.



## Conclusion
Le modèle BERT a été fine-tuné sur notre jeu de données pour l'analyse de sentiments. 
Les résultats ont été enregistrés avec MLFlow, et le modèle a montré des performances prometteuses en termes de précision.
